In [2]:
# .env 파일에 있는 환경변수를 파이썬 실행 환경으로 호출
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
OPINET_KEY = os.getenv("OPINET")

if not OPINET_KEY:
    raise ValueError("OPINET_KEY가 설정되지 않았습니다.")

In [6]:
# 시/도 코드 불러오기

import requests

url = "https://www.opinet.co.kr/api/areaCode.do"

params = {
    "code": OPINET_KEY,
    "out": "json"
}

res = requests.get(url, params=params)
data = res.json()

sido_codes = [area["AREA_CD"] for area in data["RESULT"]["OIL"]]
print(sido_codes)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '14', '15', '16', '17', '18', '19']


In [1]:
!pip install pyproj

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   -------------- ------------------------- 2.4/6.3 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 16.8 MB/s  0:00:00


In [33]:
!pip install pyproj geopy


   -------------------- ------------------- 1/2 [geopy]
   ---------------------------------------- 2/2 [geopy]



In [5]:
# 위치 입력 (주차장 정보 불러올 예정)
lat = float(input("현재 위치의 위도(lat)를 입력하세요: "))
lon = float(input("현재 위치의 경도(lon)를 입력하세요: "))

from pyproj import CRS, Transformer

# 1. 좌표계 정의 (경위도 WGS84 -> KATEC/TM128)
# KATEC(TM128)은 주로 네이버 지도 등에서 사용되던 좌표계 파라미터를 사용합니다.
wgs84_str = "epsg:4326"
katec_str = "+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,483.35,664.43,0,0,0,0"

# 2. Transformer 생성 (always_xy=True로 설정하여 경도, 위도 순서 유지)
transformer = Transformer.from_crs(wgs84_str, katec_str, always_xy=True)

# 3. 좌표 변환 실행 (경도, 위도 입력)
katec_x, katec_y = transformer.transform(lon, lat)

print(f"KATEC X: {katec_x:.2f}")
print(f"KATEC Y: {katec_y:.2f}")

KATEC X: 305384.13
KATEC Y: 544428.48


In [6]:
import requests

url = "https://www.opinet.co.kr/api/aroundAll.do"

params = {
    "code": OPINET_KEY,
    "out": "json",
    "x": katec_x,
    "y": katec_y,
    "radius": 3000,           # 반경 3km
    "prodcd": "B027",
    "sort": 2                 # 가격순
}

res = requests.get(url, params=params)

if res.status_code != 200:
    print("HTTP 오류:", res.status_code)
    exit()

data = res.json()

oil_stations = data.get('RESULT', {}).get('OIL', [])
print(oil_stations)

[{'UNI_ID': 'A0009284', 'POLL_DIV_CD': 'RTO', 'OS_NM': '진우7주유소 ㈜준우에너지', 'PRICE': 1673, 'DISTANCE': 441.9, 'GIS_X_COOR': 305291.0, 'GIS_Y_COOR': 543996.0}, {'UNI_ID': 'A0001145', 'POLL_DIV_CD': 'SKE', 'OS_NM': '대양석유(주)직영 보라매주유소', 'PRICE': 1737, 'DISTANCE': 607.2, 'GIS_X_COOR': 305321.3685, 'GIS_Y_COOR': 543824.2418}, {'UNI_ID': 'A0000433', 'POLL_DIV_CD': 'SKE', 'OS_NM': '한경석유(주)경덕주유소', 'PRICE': 1669, 'DISTANCE': 716.4, 'GIS_X_COOR': 304681.0, 'GIS_Y_COOR': 544567.0}, {'UNI_ID': 'A0009553', 'POLL_DIV_CD': 'SOL', 'OS_NM': '매일주유소', 'PRICE': 1759, 'DISTANCE': 877.1, 'GIS_X_COOR': 306033.29457, 'GIS_Y_COOR': 545018.75253}, {'UNI_ID': 'A0000371', 'POLL_DIV_CD': 'GSC', 'OS_NM': '지에스칼텍스(주) 대방주유소', 'PRICE': 1698, 'DISTANCE': 1046.9, 'GIS_X_COOR': 304963.386, 'GIS_Y_COOR': 545388.2574}, {'UNI_ID': 'A0000442', 'POLL_DIV_CD': 'SOL', 'OS_NM': '씨앤에스유통 한성주유소', 'PRICE': 1698, 'DISTANCE': 1196.3, 'GIS_X_COOR': 304968.43853, 'GIS_Y_COOR': 545551.3561}, {'UNI_ID': 'A0000410', 'POLL_DIV_CD': 'GSC', 'OS_NM'

In [22]:

# # SKE:SK에너지, GSC:GS칼텍스, HDO:현대오일뱅크, SOL:S-OIL, RTE:자영알뜰, RTX:고속도로알뜰, NHO:농협알뜰, ETC:자가상표, E1G: E1, SKG:SK가스
# for oil in oil_stations:
#     if oil.get('POLL_DIV_CD') == 'SKE':
#         brand = 'SK에너지'
#     elif oil.get('POLL_DIV_CD') == 'GSC':
#         brand = 'GS칼텍스'
#     elif oil.get('POLL_DIV_CD') == 'HDO':
#         brand = '현대오일뱅크'
#     elif oil.get('POLL_DIV_CD') == 'SOL':
#         brand = 'S-OIL'
#     elif oil.get('POLL_DIV_CD') == 'RTE':
#         brand = '자영알뜰'
#     elif oil.get('POLL_DIV_CD') == 'RTX':
#         brand = '고속도로알뜰'
#     elif oil.get('POLL_DIV_CD') == 'NHO':
#         brand = '농협알뜰'
#     elif oil.get('POLL_DIV_CD') == 'ETC':
#         brand = '자가상표'
#     elif oil.get('POLL_DIV_CD') == 'E1G':
#         brand = 'E1'
#     elif oil.get('POLL_DIV_CD') == 'SKG':
#         brand = 'SK가스'

brand_map = {
    'SKE': 'SK에너지',
    'GSC': 'GS칼텍스',
    'HDO': '현대오일뱅크',
    'SOL': 'S-OIL',
    'RTE': '자영알뜰',
    'RTX': '고속도로알뜰',
    'NHO': '농협알뜰',
    'ETC': '자가상표',
    'E1G': 'E1',
    'SKG': 'SK가스',
    'RTO': '자영알뜰'
}


# 확인용 (확인 후에는 비활성화 해주세요~!)
# for oil in oil_stations:
#     code = oil.get('POLL_DIV_CD')
#     # .get() with a default value handles unknown codes gracefully
#     brand = brand_map.get(code, '기타')
#     print(f'주유소 관리번호: {oil.get('UNI_ID')}')
#     print(f'주유소 상호명: {oil.get('OS_NM')}')
#     print(f'주유소 가격: {oil.get('PRICE')}원')
#     print(f'상표: {brand}')
#     print(f'주유소 거리: {oil.get('DISTANCE')}m')
#     print(f'주유소 X좌표: {oil.get('GIS_X_COOR')}')
#     print(f'주유소 Y좌표: {oil.get('GIS_Y_COOR')}')
#     print('\n')



In [21]:
# 주변 주유소 주소 변환

import requests
from pyproj import Transformer
from geopy.geocoders import Nominatim
import time

top_10_stations = oil_stations[:10]

# 1. 변환기 설정
# KATEC -> WGS84(경위도) 변환기
katec_str = "+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,483.35,664.43,0,0,0,0"
transformer = Transformer.from_crs(katec_str, "epsg:4326", always_xy=True)

# 주소 변환기 (Reverse Geocoding)
geolocator = Nominatim(user_agent="opinet_geocoder")

# 2. 결과 추출 및 변환
print(f"{'주유소명':<20} | {'가격':<5} | {'브랜드'} | {'주소'} | {'현재 위치에서 거리'}")
print("-" * 80)

for oil in top_10_stations:
    code = oil.get('POLL_DIV_CD')
    # .get() with a default value handles unknown codes gracefully
    brand = brand_map.get(code, '기타')
    id_ = oil.get('UNI_ID')
    name = oil.get('OS_NM')
    price = oil.get('PRICE')
    distance = oil.get('DISTANCE')
    kx = oil.get('GIS_X_COOR')
    ky = oil.get('GIS_Y_COOR')

    # A. KATEC 좌표를 위경도로 변환
    lon, lat = transformer.transform(kx, ky)

    # B. 위경도를 주소로 변환
    try:
        # 무료 API 정책 준수를 위해 1초 대기 (데이터가 많으면 필수)
        time.sleep(1)
        location = geolocator.reverse(f"{lat}, {lon}", language='ko')
        full_address = location.address if location else "주소 정보 없음"
    except Exception:
        full_address = "주소 변환 중 오류 발생"

    print(f"{name:<20} | {price:<5}원 | {brand} | {full_address} | {distance}m")

주유소명                 | 가격    | 브랜드 | 주소 | 현재 위치에서 거리
--------------------------------------------------------------------------------
진우7주유소 ㈜준우에너지        | 1673 원 | 자영알뜰 | 보라매로7길, 신대방동, 보라매동, 관악구, 서울특별시, 08709, 대한민국 | 441.9m
대양석유(주)직영 보라매주유소     | 1737 원 | SK에너지 | 보라매로, 보라매동, 관악구, 서울특별시, 08708, 대한민국 | 607.2m
한경석유(주)경덕주유소         | 1669 원 | SK에너지 | 한경석유경덕주유소, 여의대방로23길, 신길6동, 영등포구, 서울특별시, 07434, 대한민국 | 716.4m
매일주유소                | 1759 원 | S-OIL | S-OIL 매일주유소, 상도로19길, 상도3동, 동작구, 서울특별시, 06950, 대한민국 | 877.1m
지에스칼텍스(주) 대방주유소      | 1698 원 | GS칼텍스 | 여의대방로, 대방동, 동작구, 서울특별시, 07360, 대한민국 | 1046.9m
씨앤에스유통 한성주유소         | 1698 원 | S-OIL | 한성주유소, 여의대방로43가길, 신길7동, 영등포구, 서울특별시, 07360, 대한민국 | 1196.3m
지에스칼텍스㈜ 신길주유소        | 1669 원 | GS칼텍스 | 서울신길5동우체국, 65, 신길로, 신길5동, 영등포구, 서울특별시, 07425, 대한민국 | 1697.8m
CJ대한통운㈜ 신림주유소        | 1898 원 | GS칼텍스 | 대한통운직영주유소, 남부순환로, 신사동, 관악구, 서울특별시, 08762, 대한민국 | 1766.1m
HD현대오일뱅크㈜직영 관악셀프주유소  | 1684 원 | 현대오일뱅크 | (주)한화이에스직영진영주유소, 남부순환로, 미성동, 관악구, 서울특별시, 08762, 대한민국 | 1798.2m
(